## 说明

一个 RNN Transducer的示例，实现一个类似于语言翻译的序列到序列的任务；

输入为一段文本序列X，输出为另一个文本序列Y；

相当于是一个元音补全的任务：

例如： X: Hll,Wrld --> Y: Hello,World

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

import numpy
import matplotlib.pyplot as plt
from tqdm import tqdm
import unidecode
import string

In [2]:
characters = "AEIOUaeiou"
Embedding_dim = 1024
Predictor_dim = 1024
Joiner_dim = 1024
NULL_INDEX = 0


with open("war_and_peace.txt", "r") as f:
    lines = f.readlines()

len_txt = len(lines)

len_txt

62015

In [3]:
def decode_string(lst):
    return "".join(string.printable[x - 1] for x in lst)

def encode_string(line):
    return [string.printable.find(x) + 1 for x in line] # 因为0表示blank，所以要加1

## 定义数据集

In [4]:
class TxtDataset(Dataset):

    def __init__(self, lines):
        super().__init__()
        self.lines = lines

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        line = self.lines[index].replace("\n", "")
        line = unidecode.unidecode(line)  # 去除特殊字符
        x = ''.join([x for x in line if x not in characters])
        y = line

        x = encode_string(x)
        y = encode_string(y)

        # x = torch.tensor(x, dtype=torch.long)
        # y = torch.tensor(y, dtype=torch.long)

        T = torch.full(size=(1,), fill_value=len(x), dtype=torch.long)  # 时间序列的长度
        U = torch.full(size=(1,), fill_value=len(y), dtype=torch.long)  # 目标序列的长度

        return x, y, T, U
    

class CustomCollate:
    '''
    通过定义一个自定义的 Collate 类来解决 DataLoader 中不同长度数据无法堆叠的问题。
    '''

    def __call__(self, batch):
        # 拆包batch的数据
        x_list, y_list, T_list, U_list = zip(*batch)

        # 对x和y进行填充， 填充为0，0视作为blank
        x_padded = pad_sequence(x_list, batch_first=True, padding_value=0)
        y_padded = pad_sequence(y_list, batch_first=True, padding_value=0)

        # 将T和U转换为张量
        T = torch.stack(T_list)
        U = torch.stack(U_list)

        return x_padded, y_padded, T, U
    
class Collate:
    def __call__(self, batch):
        """
        batch: list of tuples (input string, output string)
        Returns a minibatch of strings, encoded as labels and padded to have the same length.
        """
        x = []; y = []
        batch_size = len(batch)
        for index in range(batch_size):
            x_, y_ , T_list, U_list= batch[index]
            x.append(x_)
            y.append(y_)

        # pad all sequences to have same length
        T = [len(x_) for x_ in x]
        U = [len(y_) for y_ in y]
        T_max = max(T)
        U_max = max(U)

        for index in range(batch_size):
            x[index] += [NULL_INDEX] * (T_max - len(x[index]))
            x[index] = torch.tensor(x[index])
            y[index] += [NULL_INDEX] * (U_max - len(y[index]))
            y[index] = torch.tensor(y[index])

        # stack into single tensor
        x = torch.stack(x)
        y = torch.stack(y)
        T = torch.tensor(T)
        U = torch.tensor(U)

        return x,y,T,U


In [5]:
string.printable

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [6]:
train_dataset = TxtDataset(lines[: round(0.9 * len_txt)])
x, y, T, U = train_dataset[0]

print(decode_string(x))
print(decode_string(y))
print(T, U)

"Wll, Prnc, s Gn nd Lcc r nw jst fmly stts f th
"Well, Prince, so Genoa and Lucca are now just family estates of the
tensor([47]) tensor([68])


In [7]:
batch_size = 128
collate  = Collate()
train_dataset = TxtDataset(lines[: round(0.9 * len_txt)])
valid_dataset = TxtDataset(lines[round(0.9 * len_txt): ])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=12, collate_fn=collate)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=12, collate_fn=collate)

In [8]:
valid_iter = iter(valid_dataloader)
x, y, T, U = next(valid_iter)
x.shape, y.shape, T.shape, U.shape

(torch.Size([128, 54]),
 torch.Size([128, 72]),
 torch.Size([128]),
 torch.Size([128]))

## 定义模型

In [9]:
class Encoder(nn.Module):
    '''
    对应声学模型
    '''
    def __init__(self, num_chars = len(string.printable) + 1):
        super().__init__()
        self.num_chars = num_chars
        self.embd = nn.Embedding(num_chars, Embedding_dim)
        self.rnn = nn.GRU(input_size=Embedding_dim, hidden_size=Embedding_dim, num_layers=3, batch_first=True, bidirectional=True, dropout=0.1)
        self.lc = nn.Linear(Embedding_dim * 2, Joiner_dim)

    def forward(self, x):
        # x input: [batch, time_step]
        x = self.embd(x)   #[batch, time_step, embdding_dim]
        x, _ = self.rnn(x)  #[batch, time_step, embdding_dim * 2]
        x = self.lc(x)  #[batch, time_step, Joiner_dim]
        return x
    
    
class PredNet(nn.Module):

    def __init__(self, num_chars = len(string.printable) + 1):
        super().__init__()
        self.embd = nn.Embedding(num_chars, Predictor_dim)
        self.rnn = nn.GRUCell(input_size=Predictor_dim, hidden_size=Predictor_dim)
        self.lc = nn.Linear(Predictor_dim, Joiner_dim)

        self.initial_state = nn.Parameter(torch.randn(Predictor_dim))  # torch.randn 生成标准正态分布
        self.start_symbol = NULL_INDEX # 0表示blank

    def forward_one_step(self, input, previous_state):
        embedding = self.embd(input) # embedding.shape: [batch， Predicotr_dim]
        state = self.rnn(embedding, previous_state) # state.shape: [batch, Predictor_dim]
        out = self.lc(state)  # out.shape: [batch, Jointer_dim]
        return out, state

    def forward(self, y):
        # y: [batch, target_num_words]
        batch_size = y.shape[0]
        U = y.shape[1]
        outs = []
        state = torch.stack([self.initial_state] * batch_size).to(y.device)  # state.shape : [batch, pred_dim]
        for u in range(U + 1):
            if u == 0:
                decoder_input = torch.tensor([self.start_symbol] * batch_size, device=y.device)  # decoder_input: [batch], 就一个维度
            else:
                # 这里训练的时候，喂给模型的是标准的 y输出
                decoder_input = y[:, u-1]  # decoder_input: [batch], 就一个维度

            out, state = self.forward_one_step(decoder_input, state)
            outs.append(out)

        out = torch.stack(outs, dim=1)  # out.shape [batch, U + 1, Jointer_dim]
        return out
    

class Jointer(nn.Module):

    def __init__(self, num_chars = len(string.printable) + 1):
        super().__init__()
        self.lc = nn.Linear(Joiner_dim, num_chars)

    def forward(self, encoder_out, pred_out):
        # encoder_out: [batch, time_step,  1, num_output]
        # pred_out: [batch, 1, target_words_len + 1, num_output]

        out = encoder_out + pred_out
        # out = torch.cat((encoder_out, pred_out), dim=-1)
        out = F.relu(out)
        out = self.lc(out)
        return out # [batch, time_step, target_words_len + 1, num_output]
    

class Transducer(nn.Module):

    def __init__(self, num_input, num_output):
        super().__init__()
        self.encoder = Encoder(num_input)
        self.pred_net = PredNet(num_output)
        self.jointer = Jointer(num_output)

        

    def forward(self, x, y, T, U):
        '''
        T: 时间序列的长度
        U: 目标序列的长度
        '''
        encoder_out = self.encoder(x)  # [batch, time_step, Joiner_dim]
        pred_out = self.pred_net(y)    # [batch, target_words_len + 1, Joiner_dim]
        joiner_out = self.jointer(encoder_out.unsqueeze(2), pred_out.unsqueeze(1)).log_softmax(3)
        return joiner_out # [batch, time_step, target_words_len + 1, num_output]
        # loss = -self.comput_forward_prob(joiner_out, T,U, y).mean()
        # return loss
    
    #  Transducer.comput_single_alignment_prob = comput_single_alignment_prob
    def greedy_search(self, x, T):
        y_batch = []
        B = len(x)
        encoder_out = self.encoder.forward(x)
        U_max = 200
        for b in range(B):
            t = 0; u = 0; y = [self.pred_net.start_symbol]; 
            predictor_state = self.pred_net.initial_state.unsqueeze(0)
            while t < T[b] and u < U_max:
                predictor_input = torch.tensor([ y[-1] ], device = x.device)
                g_u, predictor_state = self.pred_net.forward_one_step(predictor_input, predictor_state)
                f_t = encoder_out[b, t]
                h_t_u = self.jointer.forward(f_t, g_u)
                argmax = h_t_u.max(-1)[1].item()
                if argmax == 0:
                    t += 1
                else: # argmax == a label
                    u += 1
                    y.append(argmax)
            y_batch.append(y[1:]) # remove start symbol
        return y_batch
    
    def comput_forward_prob(self, joiner_out, T, U, y):
        """
        joiner_out: tensor of shape (B, T_max, U_max+1, #labels)
        T: list of input lengths
        U: list of output lengths
        y: label tensor (B, U_max+1)
        """

        B = joiner_out.shape[0]
        T_max = joiner_out.shape[1]
        U_max = joiner_out.shape[2] - 1
        log_alpha = torch.zeros(B, T_max, U_max+1, device=y.device)
        for t in range(T_max):
            for u in range(U_max+1):
                if u == 0:
                    if t == 0:
                        log_alpha[:,t,u] = 0.
                    else: # t > 0
                        log_alpha[:, t, u] = log_alpha[:, t-1, u] + joiner_out[:, t-1, 0, NULL_INDEX]

                else: # u > 0
                    if t == 0:
                        log_alpha[:, t, u] = log_alpha[:, t, u-1] + torch.gather(joiner_out[:,t, u-1], dim=1, index=y[:,u-1].view(-1,1)).reshape(-1)
                    else: # t > 0
                        log_alpha[:, t, u] = torch.logsumexp(torch.stack([
                            log_alpha[:, t-1, u] + joiner_out[:, t-1, u, NULL_INDEX],
                            log_alpha[:, t, u-1] + torch.gather(joiner_out[:,t, u-1], dim=1, index=y[:,u-1].view(-1,1) ).reshape(-1)
                        ]), dim=0)

        log_probs = []
        for b in range(B):
            log_prob = log_alpha[b, T[b]-1, U[b]] + joiner_out[b, T[b]-1, U[b], NULL_INDEX]
            log_probs.append(log_prob)
        log_probs = torch.stack(log_probs)
        return log_probs


In [10]:
rnnt = Transducer(len(string.printable) + 1, len(string.printable) + 1)
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(device)
rnnt(x, y, T, U).shape

cuda:1


torch.Size([128, 54, 73, 101])

In [11]:
class RNNTLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, logits, y, T, U):
        '''
        logits: [batch, time_step, target_words_len, num_outputs] , 预测分布
        y: target_token: [batch, target_words_len]
        T: time_step: 输入序列长度，T
        U: target_words_len: 目标序列长度 U
        '''

        batch_size, T_max, U_max, vocab_size = logits.shape
        log_alpha = torch.zeros(batch_size, T_max, U_max, device=logits.device)

        # log_alpha[t, u]：表示在时刻 t 和目标位置 u 的对数概率。该值的大小表示从输入序列的开始到当前位置的路径的概率。
        for t in range(T_max):
            for u in range(U_max):
                if u == 0:
                    if t == 0:
                        log_alpha[:, t, u] = 0.
                    else: # t > 0
                        log_alpha[:, t, u] = log_alpha[:, t-1, u] + logits[:, t-1, u, 0] #logits[:, t-1, 0, 0]： 在t-1时刻，在预测第0个单词的时候，预测是blank的概率
                else: # u > 0
                    if t == 0:
                        # torch.gather(input, dim, index) 是 PyTorch 中的一个函数，用于根据给定的 index 张量从 input 张量中按指定维度（dim）选取数据。
                        # log_alpha[:, t, u-1]: [batch, vocab_size]
                        # y[:, u-1].view(-1, 1)): [batch, 1]
                        # log_alpha[:, t, u-1]: [batch]
                        log_alpha[:, t, u] = log_alpha[:, t, u-1] + torch.gather(logits[:, t, u-1], dim=1, index=y[:, u-1].view(-1, 1)).reshape(-1)
                    else: # t > 0
                        log_alpha[:, t, u] = torch.logsumexp(torch.stack([
                            log_alpha[:, t-1, u] + logits[:, t-1, u, 0],
                            log_alpha[:, t, u-1] + torch.gather(logits[:, t, u-1], dim=1, index=y[:, u-1].view(-1, 1)).reshape(-1)
                        ]), dim=0) # 这一段写的太妙了，然后为了logsumexp可以正确使用，我们需要对两部分转移过来的数组进行stack，拼接成一个之后，指定dim就可以正确处理了。这个dim的逻辑就和sum的axis逻辑一样

        log_probs = []
        for b in range(batch_size):
            log_prob = log_alpha[b, T[b]-1, U[b]] + logits[b, T[b]-1, U[b], 0]  # 虽然padding之后所有长度都一样了，但是我们自己知道，我们需要的长度其实是T和U
            log_probs.append(log_prob)
        log_probs = torch.stack(log_probs)

        return -log_probs.mean()


In [12]:
def train(model, optimizer, loss_fn, epoch, dataloader):
    model.train()
    with tqdm(dataloader) as pbar:
        for batch_index, (x, y, T, U) in enumerate(pbar):
            x, y = x.to(device), y.to(device)
            T, U = T.to(device), U.to(device)

            optimizer.zero_grad()
            output_log_softmax = model(x, y, T, U)  # [seq_len, batch, label]
            # loss = model(x, y, T, U)  # [seq_len, batch, label]

            # target: [batch, num_words_len]
            loss = loss_fn(output_log_softmax, y, T, U)

            loss.backward()
            optimizer.step()

            loss = loss.item()

            if batch_index == 0:
                loss_mean = loss

            loss_mean = 0.1 * loss + 0.9 * loss_mean

            pbar.set_description(f'Epoch {epoch} Loss: {loss:.4f}')

            if batch_index % 15 == 0:
                model.eval()
                guesses = model.greedy_search(x, T)
                model.train()
                print("\n")
                for b in range(2):
                    print("input:", decode_string(x[b,:T[b]]))
                    print("guess:", decode_string(guesses[b]))
                    print("truth:", decode_string(y[b,:U[b]]))
                    print("")


def valid(model, optimizer, epoch, dataloader):
    model.eval()
    with tqdm(dataloader) as pbar, torch.no_grad():
        loss_sum = 0
        acc_sum = 0
        for batch_index, (data, target, input_lengths, target_lengths) in enumerate(pbar):
            data, target = data.cuda(), target.cuda()

            output = model(data)
            output_log_softmax = F.log_softmax(output, dim=-1)
            loss = F.ctc_loss(output_log_softmax, target, input_lengths, target_lengths)

            loss = loss.item()
            acc = calc_acc(target, output)

            loss_sum += loss
            acc_sum += acc

            loss_mean = loss_sum / (batch_index + 1)
            acc_mean = acc_sum / (batch_index + 1)

            pbar.set_description(f'Test : {epoch} Loss: {loss_mean:.4f} Acc: {acc_mean:.4f}')

In [13]:
optimizer = torch.optim.Adam(rnnt.parameters(), 0.0003)
loss_fn = RNNTLoss()
epochs = 15

rnnt.to(device)
for epoch in range(1, epochs + 1):
    train(rnnt, optimizer, loss_fn, epoch, train_dataloader)

Epoch 1 Loss: 384.3249:   0%|          | 1/437 [00:14<1:41:52, 14.02s/it]



input: "Wll, Prnc, s Gn nd Lcc r nw jst fmly stts f th
guess: 
truth: "Well, Prince, so Genoa and Lucca are now just family estates of the

input: Bnprts. Bt  wrn y, f y dn't tll m tht ths mns wr,
guess: 
truth: Buonapartes. But I warn you, if you don't tell me that this means war,



Epoch 1 Loss: 131.8898:   4%|▎         | 16/437 [03:22<1:30:37, 12.92s/it]



input: 
guess: 
truth: 

input: "Bt thy sy tht wr hs bn dclrd," rpld th vstr.
guess: 
truth: "But they say that war has been declared," replied the visitor.



Epoch 1 Loss: 106.9998:   7%|▋         | 31/437 [06:28<1:24:06, 12.43s/it]

...

Epoch 15 Loss: 0.9457:  96%|█████████▋| 421/437 [1:28:55<03:30, 13.18s/it]



input: th ht th drk shps f th tw wgns wth thr hrss bsd thm
guess: the het the der shaps of the tw wagens with ther hories besed them
truth: the hut the dark shapes of the two wagons with their horses beside them

input: wr dscrnbl, nd n th hllw th dyng cmpfr glmd rd. Nt
guess: were diserinle, need in the holow the dyin camier geled red. Not
truth: were discernible, and in the hollow the dying campfire gleamed red. Not



Epoch 15 Loss: 0.9314: 100%|█████████▉| 436/437 [1:32:07<00:13, 13.22s/it]



input: "Thr, hw tht dvl hts t! H's md my fc ll bldy," sd h
guess: "There, how that devil hits ut! He's med my face al bledy," said he
truth: "There, how that devil hits out! He's made my face all bloody," said he

input: n  frghtnd whspr whn th srgnt mjr hd pssd n.
guess: in a fitened whser when the seregant mar had pased on.
truth: in a frightened whisper when the sergeant major had passed on.



Epoch 15 Loss: 0.0576: 100%|██████████| 437/437 [1:32:08<00:00, 12.65s/it]


In [14]:
torch.save(rnnt, 'rnnt15epoch.pth')